# Hyperparameter Optimization

In [ ]:
%pip install datasets transformers scikit-learn pandas torch simpletransformers scipy

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
%pip install wandb -Uq
# os.environ['WANDB_START_METHOD'] = 'thread'
import wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
# os.environ['WANDB_NOTEBOOK_NAME'] = "HP_SNLI"
wandb.init(project="dissertation", name="hp_tuning_snli_new")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: darmanin-matt. Use `wandb login --relogin` to force relogin


In [5]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

In [6]:
import torch

cuda_available = torch.cuda.is_available()
cuda_available

True

In [7]:
ROOT_CSV_PATH = '/home/nli/data'
OUTPUT_PATH = '/home/nli/outputs/hyperparameter_tuning/snli'

## Steps
1. Load the datasets.
2. Rename the labels for use with the Sentence-Pair Classifier
3. Setup the model parameters
4. Run the Grid Search using W&B Sweep

In [9]:
all_data = pd.read_csv(os.path.join(ROOT_CSV_PATH, 'snli_dev_subset.csv'), delimiter=";", encoding='utf-8')
df_snli_mt_train = all_data[:20000].copy()
df_snli_mt_test = all_data[20000:25000].copy()

In [10]:
cols_to_remove = df_snli_mt_test.columns[:2]
cols_to_remove

Index(['Unnamed: 0.1', 'Unnamed: 0'], dtype='object')

In [11]:
for col in cols_to_remove:
    df_snli_mt_test.drop(col, axis=1, inplace=True)
    df_snli_mt_train.drop(col, axis=1, inplace=True)

In [12]:
df_snli_mt_train.columns = ["text_a","text_b","labels"]
df_snli_mt_test.columns =  ["text_a","text_b","labels"]

In [13]:
def map_to_num(label):
  if label == 'entailment':
    return 0
  elif label == 'contradiction':
    return 2
  else:
    return 1

def map_to_label(num):
  if num == 0:
    return "entailment"
  elif num == 2:
    return "contradiction"
  else:
    return "neutral"

In [14]:
train_labels = [map_to_num(x) for x in df_snli_mt_train['labels'].to_list()]
df_snli_mt_train['labels'] = train_labels
df_snli_mt_train["labels"] = df_snli_mt_train["labels"].astype(int)

In [15]:
eval_labels = [map_to_num(x) for x in df_snli_mt_test['labels'].to_list()]
df_snli_mt_test['labels'] = eval_labels
df_snli_mt_test["labels"] = df_snli_mt_test["labels"].astype(int)

In [16]:
df_snli_mt_train.head(5)

,text_a,text_b,labels
0,Żewġ subien żgħar jinżlu fuq għoljiet.,Żewġ subien jilagħbu hopscotch.,2
1,"Ġinnasta mara, maqbuda f'nofs l-arja waqt qabża.",Kien hemm stampa meħuda,1
2,Mara ta 'età medja liebes kappell abjad u ġakk...,Hija qed tipprova ssib kwart.,1
3,Tfal bilqiegħda ħdejn oġġetti għall-bejgħ fl-art.,it-tfal ibigħu oġġetti,1
4,kelb b'xi ħaġa ħamra f'ħalq,kelb għandu xi ħaġa,0


In [17]:
sweep_config = {
    "method": "grid",  # grid, random
    "parameters": {
        "num_train_epochs": {"values": [2,4,5]},
        "learning_rate": {"values": [2.5e-5, 5e-5, 1e-4]},
    }
}

sweep_id = wandb.sweep(sweep_config, project="dissertation")

Create sweep with ID: b64hndfv
Sweep URL: https://wandb.ai/darmanin-matt/dissertation/sweeps/b64hndfv


In [18]:
model_args = ClassificationArgs()
# model_args.num_train_epochs = 5
# model_args.learning_rate = 2.5e-5
model_args.fp16 = True
# model_args.train_batch_size = 16
# model_args.gradient_accumulation_steps = 4
# model_args.manual_seed = 4
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.no_save = True
model_args.wandb_project = 'dissertation'
model_args.max_seq_length = 512
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 5000
model_args.evaluate_during_training_verbose = True
model_args.output_dir = OUTPUT_PATH

In [19]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = 'macro')

def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = 'macro')

def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = 'macro')

In [20]:
def train():
    wandb.init()
    model = ClassificationModel("bert", "MLRS/BERTu", num_labels=3, args=model_args, use_cuda=cuda_available, sweep_config=wandb.config)
    model.train_model(df_snli_mt_train,eval_df=df_snli_mt_test, precision=precision_multiclass, f1 = f1_multiclass, recall=recall_multiclass,  acc=accuracy_score)
    model.eval_model(df_snli_mt_test, precision=precision_multiclass, f1 = f1_multiclass, recall=recall_multiclass,  acc=accuracy_score)
    wandb.join()

In [21]:
def predict_inner(model, premise, hypothesis):
    predictions, raw_outputs = model.predict([premise, hypothesis])
    return map_to_label(predictions[0])

In [ ]:
wandb.agent(sweep_id, train)